In [1]:
from symmer.chemistry import xyz_from_pubchem
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator

In [2]:
mol_name = 'H2O'

In [3]:
xyz_file = xyz_from_pubchem(mol_name)
print(xyz_file)

3
 
O	0	0	0
H	0.2774	0.8929	0.2544
H	0.6068	-0.2383	-0.7169



In [4]:
from symmer.chemistry import Draw_molecule

viewer = Draw_molecule(xyz_file,
                       width=400,
                       height=400,
                       style="stick") # change to sphere/stick
viewer.show()

## can move around in notebook!

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
from symmer.chemistry import PySCFDriver

In [6]:
basis = 'STO-3G'
convergence = 1e6
charge=0
max_hf_cycles=50
ram = 8_000
run_mp2  = True,
run_cisd = True,
run_ccsd = True,
run_fci  = True

In [7]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=charge,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

/home/tweaving/anaconda3/envs/symmer/lib/python3.8/site-packages/symmer/chemistry/fermionic_ham.py:625: UserWarning: note scf convergence threshold not very low
  warnings.warn('note scf convergence threshold not very low')


In [8]:
pyscf_obj.run_pyscf()

In [9]:
2*pyscf_obj.pyscf_hf.mol.nao

14

In [10]:
from symmer.chemistry import FermionicHamiltonian, MoleculeBuilder

In [11]:
mol = MoleculeBuilder(xyz_file)

Molecule geometry:
O	0	0	0
H	0.2774	0.8929	0.2544
H	0.6068	-0.2383	-0.7169


CISD converged?  True
FCI converged?  True
FCI converged?  True

HF energy:   -74.96444758276998
MP2 energy:  -75.00099822744197
CCSD energy: -75.01540899923558
CISD energy: -75.01477057556036
FCI energy:  -75.01553018949163


Number of qubits: 14


In [15]:
from symmer.symplectic import QuantumState
psi = mol.CI_q * QuantumState([mol.hf_array])

In [18]:
psi.dagger * mol.H_q * psi#- mol.cisd_energy

(-75.01477057556544+0j)

In [11]:
H_ferm = FermionicHamiltonian(pyscf_obj.pyscf_hf)

H_ferm.build_fermionic_hamiltonian_operator()

H_ferm.fermionic_molecular_hamiltonian

() 9.08436451197603
((0, 1), (0, 0)) -32.688952106632705
((0, 1), (2, 0)) 0.558245462869443
((0, 1), (4, 0)) 2.5402289436637416e-07
((0, 1), (6, 0)) 0.26608002538470527
((0, 1), (10, 0)) 0.313409286068878
((0, 1), (12, 0)) -1.4777973281429932e-06
((1, 1), (1, 0)) -32.688952106632705
((1, 1), (3, 0)) 0.558245462869443
((1, 1), (5, 0)) 2.5402289436637416e-07
((1, 1), (7, 0)) 0.26608002538470527
((1, 1), (11, 0)) 0.313409286068878
((1, 1), (13, 0)) -1.4777973281429932e-06
((2, 1), (0, 0)) 0.5582454628694418
((2, 1), (2, 0)) -7.581925631735148
((2, 1), (4, 0)) 3.787547388478383e-08
((2, 1), (6, 0)) -0.48849514210637696
((2, 1), (10, 0)) -1.4460616540254632
((2, 1), (12, 0)) 7.007873787145986e-06
((3, 1), (1, 0)) 0.5582454628694418
((3, 1), (3, 0)) -7.581925631735148
((3, 1), (5, 0)) 3.787547388478383e-08
((3, 1), (7, 0)) -0.48849514210637696
((3, 1), (11, 0)) -1.4460616540254632
((3, 1), (13, 0)) 7.007873787145986e-06
((4, 1), (0, 0)) 2.5402289455315885e-07
((4, 1), (2, 0)) 3.7875473853689

In [15]:
hf_state = H_ferm.hf_fermionic_basis_state
hf_state


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [16]:
hf_ket = H_ferm.hf_ket
hf_ket.shape

(16384,)

In [17]:
hfock_energy = pyscf_obj.pyscf_hf.energy_tot()
print(hfock_energy)

H_mat = H_ferm.get_sparse_ham()
hf_ket.conj().T @ H_mat @ hf_ket

-74.96386325726526


(-74.96386325726515+0j)

In [24]:
H_CI_matrix= H_ferm.get_ci_fermionic(S=0, method='CISD')

In [31]:
from symmer.utils import exact_gs_energy

e_ci, psi_ci = exact_gs_energy(H_CI_matrix)
e_ci+pyscf_obj.pyscf_hf.energy_nuc() #- pyscf_obj.pyscf_cisd.e_tot

from symmer.symplectic import QuantumState, StabilizerOp

psi = QuantumState.from_array(psi_ci).sort()

-8.071765478234738e-12

In [207]:
psi

-0.986+0.000j |11111111110000> +
 0.078+0.000j |11110011110011> +
 0.046+0.000j |11111001111001> +
 0.046+0.000j |11110110110110> +
 0.044+0.000j |11111100111100> +
 0.043+0.000j |11110011111100> +
 0.042+0.000j |11111100110011> +
 0.035+0.000j |11001111111100> +
-0.034+0.000j |11110110111001> +
-0.034+0.000j |11111001110110> +
-0.033+0.000j |11011110111100> +
 0.033+0.000j |11101101111100> +
 0.033+0.000j |11100111110110> +
 0.033+0.000j |11011011111001> +
 0.025+0.000j |11111111001100> +
-0.022+0.000j |11100111111001> +
-0.022+0.000j |11011011110110> +
 0.016+0.000j |11001111110011> +
-0.013+0.000j |11101111110100> +
 0.013+0.000j |11011111111000> +
-0.012+0.000j |11111010110101> +
-0.012+0.000j |11110101111010> +
-0.011+0.000j |11010111111010> +
-0.011+0.000j |11101011110101> +
 0.010+0.000j |11111111000011> +
 0.010+0.000j |11111101111000> +
-0.010+0.000j |11111110110100> +
 0.003+0.000j |11011110110011> +
-0.003+0.000j |11101101110011> +
-0.001+0.000j |11110111110010> +
 0.001+0.0

In [193]:
from openfermion import FermionOperator, get_sparse_operator
CI_ferm = FermionOperator()
for bvec, coeff in zip(psi.state_matrix, psi.state_op.coeff_vec):
    
    indices = np.where(np.bitwise_xor(bvec, hf_state))[0]
    
    if len(indices)==0:
        new_term = FermionOperator('', coeff)
    elif len(indices)==2:
        ferm_string = f'{indices[1]}^ {indices[0]}' 
        new_term = FermionOperator(ferm_string, coeff)
    elif len(indices)==4:
        ferm_string_1 = f'{indices[3]}^ {indices[2]}^ {indices[1]} {indices[0]}'
        ferm_string_2 = f'{indices[3]}^ {indices[2]}^ {indices[0]} {indices[1]}'
        ferm_string_3 = f'{indices[2]}^ {indices[3]}^ {indices[1]} {indices[0]}'
        ferm_string_4 = f'{indices[2]}^ {indices[3]}^ {indices[0]} {indices[1]}'
        new_term = (
            FermionOperator(ferm_string_1, -coeff/4)+
            FermionOperator(ferm_string_2, +coeff/4)+
            FermionOperator(ferm_string_3, +coeff/4)+
            FermionOperator(ferm_string_4, -coeff/4)
        )
    CI_ferm += new_term

In [194]:
QuantumState.from_array(get_sparse_operator(FermionOperator('12^ 11^ 1 2'), n_qubits=14) @ hf_ket.reshape([-1, 1]))

 1.000+0.000j |10011111110110>

In [195]:
QuantumState.from_array(hf_ket.reshape([-1, 1]))

 1.000+0.000j |11111111110000>

In [196]:
l = get_sparse_operator(CI_ferm) @ hf_ket.reshape([-1, 1])
l.conj().T @ H_mat @ l

In [197]:
l.conj().T @ H_mat @ l

array([[-75.0097147+0.j]])

In [198]:
psi_ci.conj().T @ H_mat @ psi_ci

array([[-75.01472599+0.j]])

In [199]:
pyscf_obj.pyscf_cisd.e_tot, pyscf_obj.pyscf_hf.e_tot

(-75.01472598644969, -74.96386325726526)

In [121]:
hf_state

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [47]:
psi.state_op

-0.986+0.000j XXXXXXXXXXZZZZ +
 0.078+0.000j XXXXZZXXXXZZXX +
 0.046+0.000j XXXXXZZXXXXZZX +
 0.046+0.000j XXXXZXXZXXZXXZ +
 0.044+0.000j XXXXXXZZXXXXZZ +
 0.043+0.000j XXXXZZXXXXXXZZ +
 0.042+0.000j XXXXXXZZXXZZXX +
 0.035+0.000j XXZZXXXXXXXXZZ +
-0.034+0.000j XXXXZXXZXXXZZX +
-0.034+0.000j XXXXXZZXXXZXXZ +
-0.033+0.000j XXZXXXXZXXXXZZ +
 0.033+0.000j XXXZXXZXXXXXZZ +
 0.033+0.000j XXXZZXXXXXZXXZ +
 0.033+0.000j XXZXXZXXXXXZZX +
 0.025+0.000j XXXXXXXXZZXXZZ +
-0.022+0.000j XXXZZXXXXXXZZX +
-0.022+0.000j XXZXXZXXXXZXXZ +
 0.016+0.000j XXZZXXXXXXZZXX +
-0.013+0.000j XXXZXXXXXXZXZZ +
 0.013+0.000j XXZXXXXXXXXZZZ +
-0.012+0.000j XXXXXZXZXXZXZX +
-0.012+0.000j XXXXZXZXXXXZXZ +
-0.011+0.000j XXZXZXXXXXXZXZ +
-0.011+0.000j XXXZXZXXXXZXZX +
 0.010+0.000j XXXXXXXXZZZZXX +
 0.010+0.000j XXXXXXZXXXXZZZ +
-0.010+0.000j XXXXXXXZXXZXZZ +
 0.003+0.000j XXZXXXXZXXZZXX +
-0.003+0.000j XXXZXXZXXXZZXX +
-0.001+0.000j XXXXZXXXXXZZXZ +
 0.001+0.000j XXXXXZXXXXZZZX +
 0.001+0.000j ZXXZXXXXXXZZXX +
-0.001+0

In [224]:
from symmer.chemistry.fermionic_ham import get_sign

det_i=hf_state
CI_ferm=FermionOperator()

for det_j, coeff in zip(psi.state_matrix, psi.state_op.coeff_vec):

    bit_diff = np.logical_xor(det_i, det_j)
    indices = bit_diff.nonzero()[0]
    
    if len(indices)==0:
        new_term = FermionOperator('', coeff)
    elif len(indices)==2:
        sign = get_sign(det_i, det_j, bit_diff)
        ferm_string = f'{indices[1]}^ {indices[0]}' 
        new_term = FermionOperator(ferm_string, coeff*sign)
    elif len(indices)==4:
        sign = get_sign(det_i, det_j, bit_diff)
        ferm_string_1 = f'{indices[2]}^ {indices[3]}^ {indices[1]} {indices[0]}'
        new_term = FermionOperator(ferm_string_1, coeff*sign)
    else:
        raise NotImplementedError('Excitations above doubles not currently implemented.')
    # add extra elif statements for higher order excitations
    
    CI_ferm += new_term*sign

In [223]:
pyscf_obj.pyscf_cisd.e_tot

-75.01472598644969

In [214]:
l = get_sparse_operator(CI_ferm) @ hf_ket.reshape([-1, 1])
l.conj().T @ H_mat @ l

array([[-75.01472599+0.j]])

In [216]:
# from symmer.chemistry import Draw_cube_orbital

# print('indices:', list(range(pyscf_obj.pyscf_hf.mo_coeff.shape[1])))

# index_list = [3,4]

# orb_list = Draw_cube_orbital(pyscf_obj.pyscf_hf.mol,
#                            xyz_file,  
#                            pyscf_obj.pyscf_hf.mo_coeff,
#                            index_list,
#                            width=400,
#                            height=400,
#                            style="stick") # change to sphere/stick
# for orb in orb_list:
#     orb.show()